In [8]:
import os
import pickle
import pandas as pd
from gensim.models import Word2Vec

# Train Word2Vec Embedding

Vectorise the input text data by training a new embedding using gensim's Word2Vec.

File paths

In [9]:
TRAIN_SET = os.path.join(os.pardir, 'Data', 'Tokenised_Text', 'tokenised_train.csv')
TEST_SET = os.path.join(os.pardir, 'Data', 'Tokenised_Text', 'tokenised_test.csv')

OUT_PATH = os.path.join(os.pardir, 'Data', 'Vectorised_Text')
EMBEDDING_OUT_PATH = os.path.join(os.pardir, 'Model_Build', 
                                  'Trained_Models', 'word2vec.pkl')

train_out_path = os.path.join(OUT_PATH, 'vectorised_train.csv')
test_out_path = os.path.join(OUT_PATH, 'vectorised_test.csv')

Word2Vec parameters

In [10]:
EMBEDDING_DIMENSIONS = 100
MIN_COUNT = 1
WINDOW = 5

## Load corpus to train on

In [16]:
train = pd.read_csv(TRAIN_SET)
test = pd.read_csv(TEST_SET)

corpus = pd.concat([train['comment_text'], 
                    test['comment_text']])

corpus = corpus.str.split(' ').tolist()

## Train model

Save model in case it needs uploading to Kaggle for the submission kernel

In [17]:
%%time
w2v_model = Word2Vec(
    corpus, 
    size=EMBEDDING_DIMENSIONS, 
    window=WINDOW, 
    min_count=MIN_COUNT, 
)

TypeError: 'float' object is not iterable

In [ ]:
pickle.dump(w2v_model, open(EMBEDDING_OUT_PATH, 'wb'))

## Save vectorised output